# Naive Bayes Text Classification

Using libraries:

In [10]:
import csv
import math
import nltk

Global variable:

In [11]:
dataBaseFile = "traning.csv"
dataBase = None

Support method printDataBase:

In [12]:
def printDataBase():
    for i in dataBase:
        print(i)

### Data preprocesing:

Reading .csv file:

In [13]:
def readCSVFileToArray(dataBase):
    with open(dataBaseFile, 'r') as fp:
        reader = csv.reader(fp, delimiter=',', quotechar='"')
        dataBase = [row for row in reader]
        fp.close()    
    for i in range(0,len(dataBase)):
        temp = dataBase[i][-1]
        dataBase[i][-1] = dataBase[i][0]
        dataBase[i][0] = temp
    return dataBase

Spliting:

In [14]:
def splitDataBase(dataBase):
    for i in range(0,len(dataBase)):        
        for j in range(1,len(dataBase[i])):
            tempList = str(dataBase[i][1]).split()
            dataBase[i].pop(1)
            for k in range(0,len(tempList)):
                dataBase[i].append(tempList[k])
    return dataBase

Lowercase:

In [15]:
def lowercase(dataBase):
    for i in range(0,len(dataBase)):
        for j in range(1,len(dataBase[i])):
            dataBase[i][j] = dataBase[i][j].lower()
    return dataBase

Remove STOP-WORDS:

In [16]:
def deleteSTOPWORDS(dataBase):
    nltk.download('stopwords')
    for i in range(0,len(dataBase)):
        newData = []
        for j in dataBase[i]:
            if (j not in nltk.corpus.stopwords.words('english')):
                newData.append(j)
        dataBase[i] =  newData
    return dataBase

### Naive Bayes:

In [17]:
class NaiveBayes:
    dataBase = None
    classesProb = []
    probOccurrence = []
    def __init__(self, dataBase):
        self.dataBase = dataBase
            
    def countClassesProb(self):
        for doc in self.dataBase:
            if (doc[0] not in self.classesProb):
                self.classesProb.append(doc[0])
        for i in range(0,len(self.classesProb)):
            self.classesProb[i] = 0
        for doc in self.dataBase:
            self.classesProb[int(doc[0])] = int(self.classesProb[int(doc[0])]) + 1        
        for i in range(0,len(self.classesProb)):
            self.classesProb[i] = self.classesProb[i] / len(self.dataBase)      
            
    def countProbOccurrenceInClasses(self, sample):
        self.countClassesProb()
        tableVocabulary = [[]]
        
        #generating tableVocabulary
        for i in range(0,len(self.dataBase)):
            for j in range(1,len(self.dataBase[i])):
                if (self.dataBase[i][j] not in tableVocabulary[0]):
                    tableVocabulary[0].append(self.dataBase[i][j])
        tableVocabulary[0].append("ALL")
        
        for i in range(0,len(self.classesProb)):
            tableVocabulary.append([])
        
        for k in range(1,len(tableVocabulary)):
            for q in range(0,len(tableVocabulary[0])):
                tableVocabulary[k].append(0)
        
        for i in range(0,len(self.dataBase)):
            for j in range(1,len(self.dataBase[i])):
                for k in range(0,len(tableVocabulary[0])):
                    if (tableVocabulary[0][k]==self.dataBase[i][j]):
                        tableVocabulary[int(dataBase[i][0])+1][k] += 1
                        
        print(tableVocabulary)
                        
        for i in range(1,len(tableVocabulary)):
            for j in range(0,len(tableVocabulary[i])-1):
                tableVocabulary[i][j] += 1
                
        for i in range(1,len(tableVocabulary)):
            for j in range(0,len(tableVocabulary[i])-1):
                tableVocabulary[i][len(tableVocabulary[i])-1] += tableVocabulary[i][j]
                        
        for i in range(1,len(tableVocabulary)):
            for j in range(0,len(tableVocabulary[i])-1):
                tableVocabulary[i][j] /= tableVocabulary[i][len(tableVocabulary[i])-1]  
        
        #preprosesing sample to correct x-vector
        x = [[]]
        x[0].append('0')
        x[0].append(sample)
        x = lowercase(x)
        x = splitDataBase(x)
        x = deleteSTOPWORDS(x)
        
        #countProbOccurrenceInClasses        
        for c in range(0,len(self.classesProb)):
            self.probOccurrence.append(0)
            P = 0
            for i in range(1,len(x[0])):
                for j in range(0,len(tableVocabulary[0])-1):
                    if (x[0][i] == tableVocabulary[0][j]):
                        P = P + math.log(tableVocabulary[c+1][j])
            self.probOccurrence[c] = math.log(self.classesProb[c]) + P            
        
    def fit(self):        
        accuracy = 0
        trainDataBase = []
        testDataBase = []
        for i in range(0, len(self.dataBase)):
            if ((i+1)%5 == 0):
                testDataBase.append(self.dataBase[i])
            else:
                trainDataBase.append(self.dataBase[i])
        self.dataBase = trainDataBase;
        for i in range(0, len(testDataBase)):
            sample = ""
            for j in range(1, len(testDataBase[i])):
                sample = sample + " " + testDataBase[i][j]
            if (self.predict(sample)[1]==testDataBase[i][0]):
                accuracy += 1
        accuracy = (accuracy*100)/len(testDataBase)
        accuracy = str(int(accuracy)) + "%"
        return accuracy  
    
    def predict(self, sample):
        answer = [sample]
        answer.append(str(0))
        self.countProbOccurrenceInClasses(sample)
        m = self.probOccurrence[0]
        for i in range(0,len(self.probOccurrence)):
            if (self.probOccurrence[i]>m):
                m = self.probOccurrence[i]
                answer[-1] = str(i)
        return answer

### Test model:

In [18]:
dataBase = readCSVFileToArray(dataBase)
dataBase = lowercase(dataBase)
dataBase = splitDataBase(dataBase)
dataBase = deleteSTOPWORDS(dataBase)

nb = NaiveBayes(dataBase)
nb.predict("Chinese The Chinese In Chinese Tokyo The In The Japan")
#nb.fit()

[['chinese', 'beijing', 'shanghai', 'macao', 'tokyo', 'japan', 'ALL'], [5, 1, 1, 1, 0, 0, 0], [1, 0, 0, 0, 1, 1, 0]]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Александр\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Александр\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Chinese The Chinese In Chinese Tokyo The In The Japan', '0']